# Exercice 1 :

###  1)

In [2]:
%%writefile hello_world_mpi.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

print("Hello world !")
    

Overwriting hello_world_mpi.py


In [3]:
!mpirun -n 4 python3  hello_world_mpi.py 

Hello world !
Hello world !
Hello world !
Hello world !


### 2)

In [4]:
%%writefile hello_world_mpi_r_s.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

def h_w():
    print("Hello world !",RANK,SIZE)
    
h_w()

Overwriting hello_world_mpi_r_s.py


In [5]:
!mpirun --oversubscribe -n 8 python3  hello_world_mpi.py #to use more than the real cores existing on the machine

Hello world !
Hello world !
Hello world !
Hello world !
Hello world !
Hello world !
Hello world !
Hello world !


### 3)

In [6]:
%%writefile hello_world_mpi_r_s_0.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

if RANK==0 :
    print("Hello world !",RANK,SIZE)

Overwriting hello_world_mpi_r_s_0.py


In [7]:
!mpirun -n 4 python3  hello_world_mpi.py 

Hello world !
Hello world !
Hello world !
Hello world !


# Exercice 2 :


In [7]:
%%writefile neg_value_exit.py

from mpi4py import MPI

# Initialize MPI environment
COMM = MPI.COMM_WORLD

# Get the rank of the current process
RANK= COMM.Get_rank()

while True:
    if RANK == 0:
        # Only the controller process (rank 0) reads the input
        value = int(input("Enter an integer value (or a negative integer to exit):"))
        
    else:
        value = None
        
    # Send the value to all processes
    value = COMM.bcast(value, root=0)
    
    # Exit the program in case of negative input
    if value < 0:
        break

    
    # Print the received value and the rank of the current process
    print(f"Process {RANK} got {value}")
    COMM.Barrier()

In [8]:
!mpirun -n 4 python3  neg_value_exit.py

# Exercice 3 :

### 1)

In [10]:
%%writefile broadcast_by_ring.py

from mpi4py import MPI

# Initialize MPI environment
COMM = MPI.COMM_WORLD

# Get the rank of the current process
RANK= COMM.Get_rank()

# Get the number of the machine cores
SIZE = COMM.Get_size()


while True:
    
    if RANK == 0:
        
        #Only the controller process (rank 0) reads the input
        value = int(input("Enter an integer value (or a negative integer to exit):"))
        
        COMM.send( value, dest= 1)
        
    
    else :
        
        value = COMM.recv( source= RANK-1)
        
        if RANK < SIZE-1 :
            COMM.send( value + RANK, dest= RANK + 1)
        
        
    if value < 0 :
        
        break
        
   
        
    # Print the received value and the rank of the current process
    
    print(f"Process {RANK} got {value}")
    COMM.Barrier()
        


        
        
        
        
        
        

Overwriting broadcast_by_ring.py


# Exercice 4 :

In [21]:
from mpi4py import MPI
import numpy as np

# Initialize MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Define matrix size
n = 6
m = 4

# Create matrix A on processor 0
if rank == 0:
    A = np.arange(n*m).reshape((n, m))
    print("Matrix A:\n", A)
else:
    A = None

# Define scatterv parameters
counts = [int(n*m/size)] * size
counts[0] = n*m - counts[1:]
displs = [sum(counts[:i]) for i in range(size)]

# Scatter parts of matrix A to other processors
recvbuf = np.zeros(int(n*m/size), dtype='i')
comm.Scatterv([A, counts, displs, MPI.INT], recvbuf, root=0)

# Processor 1 receives A(i,j) for i=0 to (n/2)-1 and j=m/2 to m-1
if rank == 1:
    recvbuf = recvbuf.reshape((int(n/2), int(m/2)))
    print("Processor", rank, "received:\n", recvbuf)

# Processor 2 receives A(i,j) for i=n/2 to n-1 and j=0 to (m/2)-1
if rank == 2:
    recvbuf = recvbuf.reshape((int(n/2), int(m/2)))
    print("Processor", rank, "received:\n", recvbuf)

# Processor 3 receives A(i,j) for i=n/2 to n-1 and j=m/2 to m-1
if rank == 3:
    recvbuf = recvbuf.reshape((int(n/2), int(m/2)))
    print("Processor", rank, "received:\n", recvbuf)


Matrix A:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]
 [20 21 22 23]]


TypeError: unsupported operand type(s) for -: 'int' and 'list'

# Exercice 5 :

In [8]:
from mpi4py import MPI
import numpy as np
import time

# Matrix-vector multiplication function
def matrix_vector_mult(A, x):
    m = A.shape[0]
    n = A.shape[1]
    y = np.zeros((m,))
    for i in range(m):
        for j in range(n):
            y[i] += A[i,j]*x[j]
    return y

# MPI setup
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Initialize matrix and vector
m = 1000
n = 1000
A = np.random.rand(m, n)
x = np.random.rand(n,)

# Scatter the matrix and vector
sendcounts = [m//size]*size
sendcounts[-1] += m % size
senddispls = [sum(sendcounts[:i]) for i in range(size)]
local_A = np.zeros((sendcounts[rank], n), dtype=float)
comm.Scatterv([A, sendcounts, senddispls, MPI.DOUBLE], local_A, root=0)

sendcounts = [n//size]*size
sendcounts[-1] += n % size
senddispls = [sum(sendcounts[:i]) for i in range(size)]
local_x = np.zeros(sendcounts[rank], dtype=float)
comm.Scatterv([x, sendcounts, senddispls, MPI.DOUBLE], local_x, root=0)

# Matrix-vector multiplication on each process
local_y = matrix_vector_mult(local_A, local_x)

# Gather the results
recvcounts = [m//size]*size
recvcounts[-1] += m % size
recvdispls = [sum(recvcounts[:i]) for i in range(size)]
y = np.zeros((m,), dtype=float)
comm.Gatherv(local_y, [y, recvcounts, recvdispls, MPI.DOUBLE], root=0)

# Compute the dot product on the root process
if rank == 0:
    start_time = time.time()
    y_dot = np.dot(A, x)
    dot_time = time.time() - start_time
    
    # Compare the results
    error = np.linalg.norm(y-y_dot)
    print("CPU time of parallel multiplication using {} processes is {:.6f}".format(size, dot_time))
    print("The error comparing to the dot product is :", error)

CPU time of parallel multiplication using 1 processes is 0.001518
The error comparing to the dot product is : 7747.438821026195


# Exercice 6 :

In [9]:
from mpi4py import MPI
import math

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

N = 840

# Compute the value of pi independently on each process
pi_part = 0.0
for i in range(rank, N, size):
    pi_part += 1.0/(1.0 + ((i + 0.5)/N)**2)

pi_part *= 4.0/N

# Print the partial sum computed by each process
print("Process", rank, "partial sum:", pi_part)

# Accumulate the partial sums on the controller (rank 0)
if rank == 0:
    pi = pi_part
    for i in range(1, size):
        pi += comm.recv(source=i)
else:
    comm.send(pi_part, dest=0)

# Print the final result computed by the controller
if rank == 0:
    print("Computed pi:", pi)

# Measure the time taken for the computation
comm.Barrier() # synchronize all processes
t1 = MPI.Wtime()

num_runs = 10000 # number of times to compute pi
for i in range(num_runs):
    pi_part = 0.0
    for j in range(rank, N, size):
        pi_part += 1.0/(1.0 + ((j + 0.5)/N)**2)
    pi_part *= 4.0/N

    if rank == 0:
        pi = pi_part
        for j in range(1, size):
            pi += comm.recv(source=j)
    else:
        comm.send(pi_part, dest=0)

    if rank == 0 and i == 0:
        print("Computed pi:", pi)

# Record the time taken
comm.Barrier() # synchronize all processes
t2 = MPI.Wtime()

if rank == 0:
    print("Time taken:", t2 - t1)

Process 0 partial sum: 3.1415927716925873
Computed pi: 3.1415927716925873
Computed pi: 3.1415927716925873
Time taken: 3.181356204
